In [172]:
import google.generativeai as genai
import dotenv
import os
dotenv.load_dotenv()

True

In [173]:
genai.configure(api_key=os.getenv('GEMINI_API_KEY'))
model = genai.GenerativeModel('gemini-1.5-flash')

In [174]:
def delete_files(documents):
    for document_file in documents:
        genai.delete_file(document_file.name)
        print(f'Deleted file {document_file.uri}')

In [133]:
def upload(path, n):
    chapters = []
    chapters.append(genai.upload_file(path=f"{path}/index.pdf", display_name="index"))
    print("Uploaded index")
    for i in range(1, n + 1):
        chapter_name = f"chapter_{i}"
        chapters.append(genai.upload_file(path=f"{path}/{chapter_name}.pdf", display_name=chapter_name))
        print(f"Uploaded file '{chapter_name}' as: {chapters[-1].uri}")
    return chapters

chapters = upload("./book", 5)

Uploaded index
Uploaded file 'chapter_1' as: https://generativelanguage.googleapis.com/v1beta/files/35ynckwfh0gr
Uploaded file 'chapter_2' as: https://generativelanguage.googleapis.com/v1beta/files/ysdp8wb80wsl
Uploaded file 'chapter_3' as: https://generativelanguage.googleapis.com/v1beta/files/b1tvetdr1dse
Uploaded file 'chapter_4' as: https://generativelanguage.googleapis.com/v1beta/files/jr5ehwkef7q1
Uploaded file 'chapter_5' as: https://generativelanguage.googleapis.com/v1beta/files/d18urp82ddoh


In [134]:
def view():
    # List all files
    for file in genai.list_files():
        print(f"{file.display_name}, URI: {file.uri}")
view()

chapter_5, URI: https://generativelanguage.googleapis.com/v1beta/files/d18urp82ddoh
chapter_4, URI: https://generativelanguage.googleapis.com/v1beta/files/jr5ehwkef7q1
chapter_3, URI: https://generativelanguage.googleapis.com/v1beta/files/b1tvetdr1dse
chapter_2, URI: https://generativelanguage.googleapis.com/v1beta/files/ysdp8wb80wsl
chapter_1, URI: https://generativelanguage.googleapis.com/v1beta/files/35ynckwfh0gr
index, URI: https://generativelanguage.googleapis.com/v1beta/files/aex17c8qrisr


In [175]:
def getChapter(prompt):
    chat = model.start_chat(
        history=[
            {"role": "user", "parts": f"Given a question and index of a book, specify which chapter does the question belong to. If there is any ambiguity, clarify with the user."},
            {"role": 'model', "parts": "Sure, provide me with the question and the index of the book"},
            # {"role": 'user', f"parts": "Index: {index}"},
            # {"role": 'model', "parts": "Please ask a question, and I will respond with the lesson that the question pertains to."},
        ]
    )
    response = chat.send_message([chapters[0], f"Question: {prompt}"])
    # response = chat.send_message(f"Question:{prompt}")
    print("Usage tokens: {}\n".format(response.usage_metadata.total_token_count))
    print(response.text)

In [176]:
%%time
getChapter("explain vitamin d synthesis")

Usage tokens: 4151

The question belongs to **Chapter 6: Bone Tissue and the Skeletal System** because vitamin D synthesis is a crucial aspect of bone health and development.  

It's also possible that this question might belong to **Chapter 5: The Integumentary System**  since vitamin D synthesis happens in the skin. 

To be sure, can you please specify if the question is focusing more on the bone health aspect or the skin's role in vitamin D production? 

CPU times: total: 0 ns
Wall time: 7.91 s


In [42]:
# Generate content by prompting the model with the uploaded images and a detailed question.
query = "Explain the different types of cell junctions in detail. Include the page number of the content, and if any related images are present, provide the figure number and description."
response = model.generate_content([book[0], query])
print(response.text)

Epithelial tissues are nearly completely avascular. For instance, no blood vessels cross the basement membrane to enter the
tissue, and nutrients must come by diffusion or absorption from underlying tissues or the surface. Many epithelial tissues are
capable of rapidly replacing damaged and dead cells. Sloughing off of damaged or dead cells is a characteristic of surface
epithelium and allows our airways and digestive tracts to rapidly replace damaged cells with new cells.

The cell membrane consists of two adjacent layers of phospholipids. The lipid tails of one layer face the lipid tails of the
other layer, meeting at the interface of the two layers. The phospholipid heads face outward, one layer exposed to the interior
of the cell and one layer exposed to the exterior (Figure 3.3). Because the phosphate groups are polar and hydrophilic,
they are attracted to water in the intracellular fluid. Intracellular fluid (ICF) is the fluid interior of the cell. The phosphate
groups are also a

In [ ]:
tool = {
    'function_declarations': [
        {
            'name': 'describe_with_images',
            'description': (
                'Provides a brief explanation to the question, checks for images, '
                'and retrieves image numbers and descriptions along with the page number of the content.'
            ),
            'parameters': {
                'type_': 'OBJECT',
                'properties': {
                    'question': {'type_': 'STRING'},
                    'contains_image': {'type_': 'BOOLEAN'},
                    'image_details': {
                        'type_': 'ARRAY',
                        'items': {
                            'type_': 'OBJECT',
                            'properties': {
                                'image_number': {'type_': 'NUMBER'},
                                'description': {'type_': 'STRING'}
                            },
                            'required': ['image_number', 'description']
                        }
                    },
                    'page_number': {'type_': 'NUMBER'}
                },
                'required': ['question', 'contains_image', 'page_number']
            }
        }
    ]
}

In [59]:
available_fns = ['describe_with_images']
tool_config = tool_config_from_mode("auto", available_fns)
tool_config

function_calling_config {
  mode: AUTO
  allowed_function_names: "describe_with_images"
}

In [104]:
# Generate content by prompting the model with the uploaded images and a detailed question.
prompt = "Explain what are epithelial tissues in detail. Include the page number of the content, and if any related images are present, provide the figure number and description."
response = model.generate_content([chapter[0], prompt])
print(response.text)

Epithelial tissues are sheets of cells that cover exterior surfaces of the body, lines internal cavities and passageways, and forms certain glands. It has a few distinct features. First, epithelial tissues are highly cellular, with little or no extracellular space in between cells. Second, the cells exhibit polarity with differences in structure and function between the exposed or apical facing surface of the cell and the basal surface close to the underlying body structures. Finally, epithelial tissues are nearly completely avascular; this means that no blood vessels cross the basement membrane to enter the tissue, and nutrients must come by diffusion or absorption from underlying tissues or the surface.  

Epithelial tissues are classified according to the shape of the cells and number of the cell layers formed. Cell shapes can be squamous (flattened and thin), cuboidal (boxy, as wide as it is tall), or columnar (rectangular, taller than it is wide). The number of cell layers in the 